# GBT  CoDR experiments

We implement some of the classes listed in V1.0 of the CoDR.   

* very minimal error checking
* assumed organization of PS


This should eventually reproduce Example 1 (position switching) from the GBTIDL manual. The datafile **ngc5291.fits** you need is [here](http://safe.nrao.edu/wiki/pub/GB/Data/GBTIDLExampleAndSampleData/ngc5291.fits) or locally on **/n/chara/teuben/GBT**.  Some code in here is hardcoded for this fits file, do not try it on other SDFITS files without knowing the tricks.

In [1]:
%matplotlib inline

from astropy.io import fits
from astropy import units as u
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support
from specutils import Spectrum1D, SpectrumList

from astropy.io import ascii
from astropy.nddata import StdDevUncertainty
from astropy.table import Table
from astropy.units import Unit
from astropy.wcs import WCS
from astropy.convolution import convolve, Box1DKernel

from specutils.io import get_loaders_by_extension
from specutils.io.registers import data_loader
from specutils import Spectrum1D

# Spectrum

The class that contains one spectrum

In [182]:
class Spectrum(object):
    """
    """
    def __init__(self, bintable_row):
        self.data = bintable_row['DATA']
        self.meta = bintable_row
        #
        nchan = len(self.data)        
        crval1 = bintable_row['CRVAL1']
        cdelt1 = bintable_row['CDELT1']
        crpix1 = bintable_row['CRPIX1']
        freq0  = bintable_row['RESTFREQ']
        freq = (crval1 + (np.arange(1,nchan+1) - crpix1) * cdelt1)/1e9
        self.xvals = freq
        

# SDFITSLoad

This is the class that loads an SDFITS file. Normally not called by users, but by classes such as GBTLoadPS()



In [183]:
class SDFITSLoad(object):
    """
    container for a bintable from a selected HDU
    """
    def __init__(self, filename, hdu=1):
        """
        """
        print("==SDFITSLoad %s" % filename)
        self.filename = filename
        self.bintable = None
        self.hdu = fits.open(filename)
        self.header = self.hdu[0].header
        self.load(hdu)
    def load(self, hdu=1):
        """
        for given hdu make this bintable available
        """
        self.bintable = self.hdu[hdu]
        self.header2  = self.bintable.header
        self.data2    = self.bintable.data
        src = np.unique(self.data2[:]['OBJECT'])
        scan = np.unique(self.data2[:]['SCAN'])
        ncols = self.header2['NAXIS1']
        nrows = self.header2['NAXIS2']
        #
        print("File:     %s   HDU %d" % (self.filename, hdu))
        print("BINTABLE: %d rows x %d cols" % (nrows,ncols))
        print("Sources: ",src)
        print("Scans:   ",scan)
        
        
s = SDFITSLoad('ngc5291.fits')

==SDFITSLoad ngc5291.fits
File:     ngc5291.fits   HDU 1
BINTABLE: 352 rows x 131458 cols
Sources:  ['NGC5291']
Scans:    [51 52 53 54 55 56 57 58]


# GBTLoad

This is the base class from which we derive all GBTLoad* subclasses that can load and calibrate spectra. It can also be used to load all the spectra, but there is no structure defined, e.g. to check and guide calibration.

In [184]:
class GBTLoad(object):
    def __init__(self, filename):
        print("==GBTLoad %s" % filename)
        self.filename = filename
        self.s = SDFITSLoad(filename)
        self.nrows = self.s.header2['NAXIS2']
        self.sp = np.empty(self.nrows, dtype=Spectrum)
        for i in range(self.nrows):
            self.sp[i] = Spectrum(self.s.data2[i])
    def __len__(self):
        return self.nrows
    def __getitem__(self, index):
        return self.sp[index]
    def __repr__(self):
        return self.filename
        
g = GBTLoad('ngc5291.fits')

==GBTLoad ngc5291.fits
==SDFITSLoad ngc5291.fits
File:     ngc5291.fits   HDU 1
BINTABLE: 352 rows x 131458 cols
Sources:  ['NGC5291']
Scans:    [51 52 53 54 55 56 57 58]


In [188]:
print(len(g))
a=g[1]
print(len(a.xvals))

352
32768


# GBTLoadPS

This is the class that loads and calibrates Position Switched (PS) data

In [ ]:
def uniq(seq):
    """ from http://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-in-python-whilst-preserving-order """
    seen = set()
    seen_add = seen.add
    return [ x for x in seq if x not in seen and not seen_add(x)]


In [151]:
class GBTLoadPS(GBTLoad):
    """
    
    """
    def __init__(self, filename):
        """
        Load SDFITS into a PS structure
        - should also allow SDFITSLoad() list
        - should also allow filename list
        - all assumed hdu=1
        """
        def ushow(name):
            uname = np.unique(d[:][name])
            print('uniq',name,uname)
            return uname
        def wh(name,val):
            print('no')
                              
        self.filename = filename
        self.s = SDFITSLoad(filename)
        d = self.s.data2
        ushow('OBJECT')
        scans = ushow('SCAN')
        pols = ushow('SAMPLER')
        ushow('SIG')
        ushow('CAL')
        ushow('PROCSEQN')
        ushow('PROCSIZE')
        ushow('OBSMODE')
        

p = GBTLoadPS('ngc5291.fits')

==SDFITSLoad ngc5291.fits
File:     ngc5291.fits   HDU 1
BINTABLE: 352 rows x 131458 cols
Sources:  ['NGC5291']
Scans:    [51 52 53 54 55 56 57 58]
uniq OBJECT ['NGC5291']
uniq SCAN [51 52 53 54 55 56 57 58]
uniq SAMPLER ['A13' 'A9']
uniq SIG ['T']
uniq CAL ['F' 'T']
uniq PROCSEQN [1 2]
uniq PROCSIZE [2]
uniq OBSMODE ['OnOff:PSWITCHOFF:TPWCAL' 'OnOff:PSWITCHON:TPWCAL']


In [171]:
#   FS example (should not load)
ex2 = GBTLoadPS('W3OH.fits')

==SDFITSLoad W3OH.fits
File:     W3OH.fits   HDU 1
BINTABLE: 480 rows x 65922 cols
Sources:  ['W3OH']
Scans:    [79 80 81 82 83]
uniq OBJECT ['W3OH']
uniq SCAN [79 80 81 82 83]
uniq SAMPLER ['A10' 'A13' 'A14' 'A9']
uniq SIG ['F' 'T']
uniq CAL ['F' 'T']
uniq PROCSEQN [1]
uniq PROCSIZE [1]
uniq OBSMODE ['Track:FSWITCH:FSW01']


In [172]:
# NOD example (should not load)
ex3 = GBTLoadPS('IC1481.fits')

==SDFITSLoad IC1481.fits
File:     IC1481.fits   HDU 1
BINTABLE: 512 rows x 33154 cols
Sources:  ['IC1481']
Scans:    [182 183 184 185 186 187 188 189]
uniq OBJECT ['IC1481']
uniq SCAN [182 183 184 185 186 187 188 189]
uniq SAMPLER ['A1' 'A2' 'B3' 'B4' 'C5' 'C6' 'D7' 'D8']
uniq SIG ['T']
uniq CAL ['F' 'T']
uniq PROCSEQN [1 2]
uniq PROCSIZE [2]
uniq OBSMODE ['Nod:NONE:TPWCAL']


In [173]:
d = p.s.data2

wh1 = d['SAMPLER'] == 'A13' 
wh2 = d['SAMPLER'] == 'A9'
wh3 = d['CAL'] == 'T'
wh4 = d['CAL'] == 'F'

In [174]:
wh_1 = np.bitwise_and(wh1,wh3)


352

In [162]:
np.bitwise_and?
